In [ ]:
from preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split

In [ ]:
parent_dir = './Data/In-lab/'
paths = sorted(list(os.listdir(parent_dir)))
paths = np.array(paths)
np.random.shuffle(paths)
training_paths = paths[:-4]
validation_paths = paths[-4:]
interval_duration = 0.25
vec_size = int(15_000 * interval_duration)


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def return_dataset(paths):
    data = []

    for path in tqdm(paths):
        temp = get_data(parent_dir+path, interval_duration=interval_duration)
        dataX = temp[:, :vec_size].astype(float)    
        remaining = temp[:, vec_size:]
        # z score normalization
        dataX -= dataX.mean()
        dataX /= np.std(dataX)
        temp = np.concatenate([dataX, remaining], axis=1)
        data.append(temp)
    data = np.concatenate(data, axis=0)
    dataX = data[:, :vec_size].astype(float)
    dataY = data[:, -1].astype(float)
    dataX, dataY = unison_shuffled_copies(dataX, dataY)
    return dataX, dataY


trainX, trainY = return_dataset(training_paths)
valX, valY = return_dataset(validation_paths)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

trainX = torch.Tensor(trainX, device=device)
trainY = torch.Tensor(trainY, device=device)

train_dataset = TensorDataset(trainX, trainY)
train_dataloader = DataLoader(train_dataset)

valX = torch.Tensor(valX, device=device)
valY = torch.Tensor(valY, device=device)

val_dataset = TensorDataset(valX, valY)
val_dataloader = DataLoader(val_dataset)

In [ ]:
from torch_model import TSNetwork

model = TSNetwork()

model.to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
epochs = 100

for epoch in tqdm(range(epochs)):
    running_loss = 0
    last_loss = 0
    
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch}: Loss: {running_loss}")